# CLogit-SUE: an illustration of fixed point formulation with MSA as the solution algorithm.
Reference: Zhou, Z., Chen, A., & Bekhor, S. (2012). C-logit stochastic user equilibrium model: formulations and solution algorithm. Transportmetrica, 8(1), 17-41.

In [348]:
using DelimitedFiles:readdlm

## Problem setting.

<img src="data/Caliper_net.png" width="800">

In [349]:
netFile = "data/network.csv"
csvdata = readdlm(netFile, ',', header=true)
data = csvdata[1]
header = csvdata[2]

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

In [350]:
display(header)
display(data)

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

7×10 Matrix{Float64}:
 1.0  2.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 1.0  4.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  3.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  5.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 3.0  6.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 4.0  5.0  50.0  20.0  10.0  0.15  4.0  60.0  4.0  1.0
 5.0  6.0  50.0  20.0  10.0  0.15  4.0  60.0  2.0  1.0

In [351]:
links = []
for l in zip(round.(Int, data[:,1]), round.(Int, data[:,2]))
    push!(links, l)
end
cap = data[:, 3]
length = data[:, 4]
fft = data[:, 5]
alpha = data[:, 6]
beta = data[:, 7]
dem = 100.0

100.0

In [352]:
path_length = [60.0; 60.0; 60.0]
overlap = zeros(Float64, 3, 3)
overlap[1, 2] = 20
overlap[1, 3] = 0
overlap[2, 1] = 20
overlap[2, 3] = 20
overlap[3, 1] = 0
overlap[3, 2] = 20
overlap

3×3 Matrix{Float64}:
  0.0  20.0   0.0
 20.0   0.0  20.0
  0.0  20.0   0.0

In [353]:
# BPR function
function BPR(cap, fft, alpha, beta,  flow)
    return fft * (1 + alpha * (flow/cap) ^ beta)
end

BPR (generic function with 1 method)

## Flow-independent commonality factor.

In [354]:
# Flow-independent commonality factor for C-Logit model. Use length to calculate the commonality factor.
temp = [0.0, 0.0, 0.0]
for i in 1:size(overlap)[1]
    for j in 1:size(overlap)[2]
        temp[i] += overlap[i, j] / (sqrt(path_length[i]) * sqrt(path_length[j]))
    end
end
coeff = 10
cf = coeff * log.(temp)

3-element Vector{Float64}:
 -10.986122886681098
  -4.054651081081644
 -10.986122886681098

In [355]:
# C-Logit probability
function CLogit(cost, cf)
    theta = 1.0
    util = -(cost + cf)
    return exp.(theta * util) / sum(exp.(theta * util))
end

CLogit (generic function with 1 method)

## Formulation
It is a fixed point problem with respect to path flow. 
$$
\mathbf{f} = F(\mathbf{f}) \Rightarrow \mathbf{f = q * Pr(-\Delta^{T}t(\Delta f))}
$$

Alternatively, we can also write it as a fixed point problem with respect to link flow.
$$
\mathbf{x} = G(\mathbf{x}) \Rightarrow \mathbf{x = \Delta q * Pr(-\Delta^{T}t(x))}
$$

We use MSA method to solve this fixed point problem with respect to path flow. 

In [356]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = dem * CLogit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f), cf)
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [29.488980514315916, 3.4761026771776984, 67.03491680850638]
f     = [29.488983103518574, 3.47609895880455, 67.03491793767664]
x     = [32.96508206232313, 67.03491793767664, 29.488983103518574, 3.47609895880455, 29.488983103518574, 67.03491793767664, 70.5110168964812]


In [357]:
# Let's also do it link flow fixed point problem.
# Initial value
f = [0; 100; 0]
x = Δ*f
x_aux = []
for iter in 1:10000
    x_aux = Δ * dem * CLogit(Δ' * BPR.(cap, fft, alpha, beta, x), cf)
    x = (x_aux  + iter * x) / (1 + iter)
end
println("x_aux = $x_aux")
println("x     = $x")

x_aux = [32.96507938503218, 67.0349206149678, 29.48898924258718, 3.476090142445001, 29.48898924258718, 67.0349206149678, 70.5110107574128]
x     = [32.96508597226057, 67.03491402773939, 29.488974137969002, 3.4761118342915895, 29.488974137969002, 67.03491402773939, 70.51102586203058]


## Flow-dependent commonality factor.

In [358]:
# Flow-dependent commonality factor for C-Logit model. Use time to calclualte the commonality factor.
function getOverlap(t)
    overlap = zeros(Float64, 3, 3)
    overlap[1, 2] = t[1]
    overlap[1, 3] = 0.0
    overlap[2, 1] = t[1]
    overlap[2, 3] = t[7]
    overlap[3, 1] = 0.0
    overlap[3, 2] = t[7]
    return overlap
end

function commonality(overlap)
    temp = [0.0, 0.0, 0.0]
    for i in 1:size(overlap)[1]
        for j in 1:size(overlap)[2]
            temp[i] += overlap[i, j] / (sqrt(path_length[i]) * sqrt(path_length[j]))
        end
    end
    coeff = 10
    cf = coeff * log.(temp)
    return cf
end

commonality (generic function with 1 method)

In [359]:
# C-Logit probability
function CLogit(pcost, cf)
    theta = 1.0
    util = -(pcost + cf)
    return exp.(theta * util) / sum(exp.(theta * util))
end

CLogit (generic function with 1 method)

In [360]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = dem * CLogit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f), commonality(getOverlap(BPR.(cap, fft, alpha, beta, Δ * f))))
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [27.669837117482604, 4.002320694601948, 68.32784218791545]
f     = [27.669837793017507, 4.002319596655362, 68.32784261032714]
x     = [31.67215738967287, 68.32784261032714, 27.669837793017507, 4.002319596655362, 27.669837793017507, 68.32784261032714, 72.33016220698251]


In [361]:
# Let's also do it link flow fixed point problem.
# Initial value
f = [0; 100; 0]
x = Δ*f
x_aux = []
for iter in 1:10000
    x_aux = Δ * dem * CLogit(Δ' * BPR.(cap, fft, alpha, beta, x), commonality(getOverlap(BPR.(cap, fft, alpha, beta, x))))
    x = (x_aux  + iter * x) / (1 + iter)
end
println("x_aux = $x_aux")
println("x     = $x")

x_aux = [31.672156670887915, 68.3278433291121, 27.669838942517284, 4.002317728370629, 27.669838942517284, 68.3278433291121, 72.33016105748273]
x     = [31.672158333595814, 68.3278416664044, 27.669836283467397, 4.002322050128581, 27.669836283467397, 68.3278416664044, 72.33016371653238]
